Import dependencies

In [1]:
import os
import pandas as pd
import numpy as np

Check directory and file names

In [2]:
os.chdir(r'C:\Users\billk\Desktop\to_annotate')
print(os.getcwd())

C:\Users\billk\Desktop\to_annotate


Open csv

In [3]:
data = pd.read_csv("Combined_labels.csv")
data.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,1.jpg,960,960,1,677,373,761,401
1,10.jpg,960,960,1,500,468,553,537
2,100.jpg,661,661,1,403,417,479,474
3,100.jpg,661,661,1,441,499,548,563
4,100.jpg,661,661,1,6,563,85,593


In [8]:
data.shape

(455, 8)

Omit object class 2 (out of focus) and 3 (10X) to keep class 1 (40X in focus images)

In [59]:
data = data[data['class'] == 1]
data.shape

(324, 8)

Assign modeling groups

In [65]:
data2 = data.drop_duplicates(subset = ['filename'])

In [74]:
np.random.seed(42)
wookie = np.random.randint(1, 10000, size=len(data2['filename']))/10000
groups = np.where(wookie <= 0.2, (np.where(wookie <= 0.1, 'VALIDATION', 'TESTING')),'TRAINING')

In [89]:
traingroup = []

for filename in data['filename']:
    traingroup.append(groups[data2['filename'] == filename][0])
    
len(traingroup)

324

tflite_model_maker csv structure

In [90]:
filepath = []
object_class = []
xmin = []
ymin = []
TRX = []
TRY = []
xmax = []
ymax = []
BLX = []
BLY = []

fill vectors

In [91]:
filepath = (list(data['filename'].values))
object_class = (list(data['class'].values))
xmin = (list(data['xmin'].values/data['width']))
ymin = (list(data['ymin'].values/data['height']))
xmax = (list(data['xmax'].values/data['width']))
ymax = (list(data['ymax'].values/data['height']))
TRX = ([''] *len(data['filename']))
TRY = ([''] *len(data['filename']))
BLX = ([''] *len(data['filename']))
BLY = ([''] *len(data['filename']))

Build tf_lite csv

In [92]:
tensorflow = {
    'traingroup': traingroup, 
    'filepath': filepath, 
    'object_class': object_class, 
    'xmin': xmin, 
    'ymin': ymin, 
    'TRX': TRX, 
    'TRY': TRY, 
    'xmax': xmax, 
    'ymax': ymax, 
    'BLX': BLX, 
    'BLY': BLY}

In [93]:
df=pd.DataFrame(tensorflow)
df

,traingroup,filepath,object_class,xmin,ymin,TRX,TRY,xmax,ymax,BLX,BLY
0,TRAINING,1.jpg,1,0.705208,0.388542,,,0.792708,0.417708,,
1,VALIDATION,10.jpg,1,0.520833,0.487500,,,0.576042,0.559375,,
2,TRAINING,100.jpg,1,0.609682,0.630862,,,0.724660,0.717095,,
3,TRAINING,100.jpg,1,0.667171,0.754917,,,0.829047,0.851740,,
4,TRAINING,100.jpg,1,0.009077,0.851740,,,0.128593,0.897126,,
...,...,...,...,...,...,...,...,...,...,...,...
319,TRAINING,a7.jpg,1,0.435065,0.779221,,,0.551948,0.954545,,
320,TRAINING,a7.jpg,1,0.305195,0.396104,,,0.435065,0.571429,,
321,TRAINING,a7.jpg,1,0.097403,0.019481,,,0.240260,0.246753,,
322,TRAINING,a8.jpg,1,0.189189,0.464865,,,0.545946,0.659459,,


In [96]:
df.to_csv('data.csv', index=None)